In [2]:
from dotenv import load_dotenv
import os
from influxdb_client import InfluxDBClient, QueryApi
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from typing import Annotated, Sequence, Tuple, TypedDict, Union
from typing_extensions import TypedDict
from langchain_core.messages import (
    BaseMessage,
    FunctionMessage,
    HumanMessage,
)
import operator

load_dotenv()

influxdb_url = os.getenv("INFLUXDB_URL")
influxdb_token = os.getenv("INFLUXDB_TOKEN")
influxdb_org = os.getenv("INFLUXDB_ORG")
api_key = os.getenv("OPENAI_API_KEY")

client = InfluxDBClient(
    url=influxdb_url,
    token=influxdb_token,
    org=influxdb_org,
)

query_api = client.query_api()

In [5]:
def create_agent(llm, tools, system_message: str):
    # 에이전트를 생성합니다.
    functions = [format_tool_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join(
        [tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

In [8]:
def influx_tool(
    query: Annotated[str, "The complete Flux query to execute. This should be a valid Flux query language string."],
):
    """
    Execute a Flux query on InfluxDB.
    
    Args:
        query: The complete Flux query to execute. This should be a valid Flux query language string.
              Example: 'from(bucket: "P1-A_status") |> range(start: -1h) |> filter(fn: (r) => r._measurement == "available")'
        
    Returns:
        The query results formatted as a string
    """
    try:
        tables = query_api.query(query, org=influxdb_org)
        
        # Convert to a more usable format
        results = []
        for table in tables:
            for record in table.records:
                results.append(dict(record.values))
                
        if not results:
            return "No results found for the query."
        
        # Format the results for readability
        formatted_results = ""
        for i, record in enumerate(results[:10]):
            formatted_results += f"Record {i+1}:\n"
            for key, value in record.items():
                formatted_results += f"  {key}: {value}\n"
            formatted_results += "\n"
        
        if len(results) > 10:
            formatted_results += f"... and {len(results) - 10} more records"
            
        return formatted_results
    except Exception as e:
        return f"Error querying InfluxDB: {str(e)}"

In [16]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str